In [30]:
import requests, sys
import json
import pandas as pd

In [ ]:
accession = "P0DJI9"
variant_index = 'NCBI Reference SNP (rs) Report ALPHA'

In [69]:
def get_protein_variation(accession: str) -> tuple[dict, pd.DataFrame]:
    requestURL = f"https://www.ebi.ac.uk/proteins/api/variation?offset=0&size=-1&accession={accession}"
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    responseBody = r.text

    data = json.loads(responseBody)[0]

    features = pd.DataFrame(data.pop('features'))
    return data, features

In [71]:
def get_ref_aa_sequence(peptide_sequence: str, feature: dict) -> str:
    """
    Get the reference amino acid for a given feature.
    
    Args:
        sequence (str): The full protein sequence.
        feature (dict): A feature dictionary containing 'begin' and 'end'.
        
    Returns:
        str: The reference amino acid or None if not applicable.
    """
    start = int(feature.get('begin'))
    end = int(feature.get('end'))
    
    return peptide_sequence[start-1:end]

In [93]:
def get_position(feature: dict) -> str:
    """
    Get the position of a feature in the protein sequence.

    Args:
        feature (dict): A feature dictionary containing 'begin' and 'end'.

    Returns:
        str: The position in the format "start-end".
    """
    begin = feature.get('begin')
    end = feature.get('end')
    if begin == end:
        return str(begin)
    return f"{feature.get('begin')}-{feature.get('end')}"

In [6]:
def get_variant_id(feature: dict) -> str:
    """
    Get the variant ID from a feature.

    Args:
        feature (dict): A feature dictionary.

    Returns:
        str: The variant ID or None if not applicable.
    """
    xrefs = feature.get('xrefs', [])
    for xref in xrefs:
        if xref.get('id').startswith('rs'):
            return xref.get('id')
    return ''

In [15]:
def get_frequency(variant:str) -> str:
    if not variant:
        return ''
        
    try:
        freq_url = f"https://www.ncbi.nlm.nih.gov/snp/{variant}/download/frequency"
        r = requests.get(freq_url, headers={ "Accept" : "application/json"})
        if not r.ok:
            r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching frequency data for variant {variant}: {e}")
        return ''

In [ ]:
def parse_frequency_reponse(responseBody: str) -> tuple[dict, pd.DataFrame]:
    """
    Parse the frequency response body.

    Args:
        responseBody (str): The response body as a string.

    Returns:
        dict: Parsed JSON data.
    """
    if responseBody == '':
        return {}, pd.DataFrame()

    lines = responseBody.splitlines()
    n_lines = len(lines)
    i = 0

    metadata = {}
    header = []
    rows = []

    while i < n_lines:
        line = lines[i]
        tokens = line.split('\t')

        if len(tokens) == 2:
            key = tokens[0].strip('# ')
            value = tokens[1].strip()
            metadata[key] = value
        elif len(tokens) > 2:
            if tokens[0].startswith('#'):
                header = [t.strip('# ') for t in tokens]
            else:
                row = list(map(lambda x: 'NA' if x == "" else x, tokens))
                rows.append(row)
        elif len(tokens) == 1:
            pass
        else:
            print(f"Unexpected line format at line {i}: {line}")
            sys.exit(1)
        i += 1
        continue
    
    df = pd.DataFrame(rows, columns=header)
    df[variant_index] = metadata.get(variant_index)
    return metadata, df

In [75]:
def combine_alternative_sequences(df):
    if 'alternativeSequence' not in df.columns:
        return df
    return (
        df.groupby(['begin', 'end', 'variant_id'], dropna=False, as_index=False)
        .agg({col: (lambda x: ','.join(x.astype(str).unique())) if col == 'alternativeSequence' else 'first'
              for col in df.columns})
    )

In [88]:
data, features_df = get_protein_variation(accession)
features_df['variant_id'] = features_df.apply(get_variant_id, axis=1)
features_df['variation_position'] = features_df.apply(get_position, axis=1)
features_df['ref_aa'] = features_df.apply(lambda x: get_ref_aa_sequence(data.get('sequence'), x), axis=1)
features_df = combine_alternative_sequences(features_df)

In [78]:
results = list(zip(*[parse_frequency_reponse(get_frequency(variant)) for variant in features_df['variant_id']]))

Error fetching frequency data for variant rs1559599007: 500 Server Error: Internal Server Error for url: https://www.ncbi.nlm.nih.gov/snp/rs1559599007/download/frequency


In [79]:
metadata_df = pd.DataFrame(results[0])
frequencies_df = pd.concat(results[1])

In [89]:
merged = pd.concat([features_df, metadata_df], axis=1)
final_merged = pd.merge(merged, frequencies_df, on=variant_index, how='outer')


In [90]:
final_merged[['Ref Allele NA', 'Ref Allele Prob']] = final_merged['Ref Allele'].str.split('=', n=1, expand=True)
final_merged[['Alt Allele NA', 'Alt Allele Prob']] = final_merged['Alt Allele'].str.split('=', n=1, expand=True)

In [91]:
final_merged_filtered = final_merged.loc[:, final_merged.isna().mean() < 0.8]

In [92]:
final_merged.to_csv(f"{accession}_protein_variation.csv", index=False, sep='\t')